# Cluster-based Vote Count Prediction for new images

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn

## Loading Data

In [5]:
# Loading IncV3 latent features sim. matrix and votes
SIM_MX_FILE_PATH = os.path.join('..', '..', 'results', 'matrices', 'incv3_feats_euclid_sim_matrix.csv')
VOTES_FILE_PATH = os.path.join('..', '..', 'results', 'votes_summary.csv')

In [ ]:
# lOADING 
FEATS_FILE_PATH = os.path.join('..', '..', 'results', 'features', 'incv3_feats.csv')

#### Data (Sim. Matrix between images)

In [6]:
sim_mx_df = pd.read_csv(SIM_MX_FILE_PATH, index_col=0)
sim_mx_df.head(3)

FileNotFoundError: [Errno 2] File b'../../results/matrices/incv3_feats_euclid_sim_matrix.csv' does not exist: b'../../results/matrices/incv3_feats_euclid_sim_matrix.csv'

#### Votes

In [ ]:
votes_df = pd.read_csv(VOTES_FILE_PATH, index_col=0)
votes_df.head(3)

Here's a sanity check for vote proportion in our the dataset. In the original XAI-CBR paper, vote proportion was like this:
- IG: 45%
- XRAI: 30%
- LIME: 18%
- ANCHOR: 7%

Also, IG was the most voted technique, at least by hard voting aggregation, with a majority of 62% images.


In [ ]:
votes_df[['ig','lime','xrai','anchor']].sum() / 2867

There's a slight imbalance of these proportions with respect to ones presented in the paper. It seems like some votes from XRAI and ANCHOR techniques drifted out to the IG technique. We'll check this out later, this should not be of great importance in the experiments of this notebook.

### Data Preprocessing

In [ ]:
X = sim_mx_df.values # Values from sim. matrix
X_names = sim_mx_df.index.values # Names of every image
y = votes_df.values[:, :4] # Vote count for each imae
best = votes_df.values[:, -1] # Most voted technique for each image

In [ ]:
print(X.shape, X_names.shape, y.shape, best.shape)

#### Instance deletion
Stratified Subsampling cannot be performed onto the dataset because only one instance is best explained with ANCHOR. Due to the very small importance of that instance in the dataset, we will continue without that instance (i.e. we will find that instance and remove it from the dataset).

In [ ]:
# At what index is the anchor instance located?
anchor_idxs = np.argwhere(best == 'anchor')[0]
anchor_idxs

In [ ]:
# What's the name of that image and its associated technique?
X_names[anchor_idxs], best[anchor_idxs]

In [ ]:
# Delete that instance from all data partitions (X, y, etc.)
X = np.delete(X, anchor_idxs, axis=0)
X = np.delete(X, anchor_idxs, axis=1) # Twice in sim. matrix (both rows and columns)
X_names = np.delete(X_names, anchor_idxs, axis=0)
y = np.delete(y, anchor_idxs, axis=0)
best = np.delete(best, anchor_idxs, axis=0)

In [ ]:
print(X.shape, X_names.shape, y.shape, best.shape)

## Splitting and Fold Creation

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit as SSS
from sklearn.model_selection import ShuffleSplit as SS

In [ ]:
# Change this constant to toogle stratified sampling on/off
STRATIFIED = True

In [ ]:
# Perform split
splitter = None
if STRATIFIED: splitter = SSS(n_splits=5, test_size=0.2, random_state=42)
else: splitter = SS(n_splits=5, test_size=0.2, random_state=42)
splits = splitter.split(X, best)
splits = list(splits)

In [ ]:
splits[0]

## Clustering

In [ ]:
clusterable_params = []

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [ ]:
def get_sim_mx_subset(sim_mx_values, filter_idxs):
    return sim_mx_values.take(filter_idxs, axis=0).take(filter_idxs, axis=1)

In [ ]:
def fit_dbscan_sim_mx(data, min_samples, eps_values, 
               min_no_clusters=5, max_no_clusters=np.inf,
               min_clust_instances=None, min_clust_instances_pct=0.85,
               max_clust_instances=np.inf):
    # Condition precalculation
    if min_clust_instances_pct: # If % was defined
        min_clust_instances = round(data.shape[0] * min_clust_instances_pct)
    elif not min_clust_instances: # Else, if nominal amount was not specified
        min_clust_instances = 100
    # Code
    scores, clusters, instances = [], [], []
    for m in min_samples:
        row_scores, row_clusters, row_instances = [], [], []
        for e in eps_values:
            db = DBSCAN(min_samples=m, eps=e, metric='precomputed').fit(data)
            # Get only non anomalous instances and indices
            non_a = db.labels_ != -1 # [False, ..., False] if all are outliers
            non_a_idxs = np.argwhere(non_a==True)
            non_a_idxs = non_a_idxs.reshape(non_a_idxs.shape[0])
            # Calculate conditions
            n_clusters = len(np.unique(db.labels_[non_a])) # 0 if all are outliers
            n_instances = len(db.labels_[non_a]) # 0 if all are outliers
            # Apply conditions (why does it output NaN and not None?)
            valid_n_clusters = n_clusters >= min_no_clusters and n_clusters <= max_no_clusters
            valid_n_cl_instances = n_instances >= min_clust_instances and n_instances <= max_clust_instances
            if (valid_n_clusters and valid_n_cl_instances):
                non_a_data = get_sim_mx_subset(data, non_a_idxs)
                score = silhouette_score(non_a_data, db.labels_[non_a], metric='precomputed')
            else:
                score = None
            # Store results
            row_scores.append(score)
            row_clusters.append(n_clusters)
            row_instances.append(n_instances)
        # Store row results
        scores.append(row_scores)
        clusters.append(row_clusters)
        instances.append(row_instances)
    # Prepare and return values
    ms_axis = pd.Index(min_samples, name='Min_samples')
    eps_axis = pd.Index(eps_values, name='Epsilon')
    df_scores = pd.DataFrame(scores, index=ms_axis, columns=eps_axis)
    df_clusters = pd.DataFrame(clusters, index=ms_axis, columns=eps_axis)
    df_instances = pd.DataFrame(instances, index=ms_axis, columns=eps_axis)
    return df_scores, df_clusters, df_instances

In [ ]:
def print_results(m, eps, scores_df, instances_df, clusters_df):
    score = round(scores_df.loc[m][eps], 4)
    instances = instances_df.loc[m][eps]
    clusters = clusters_df.loc[m][eps]
    print(f'DBSCAN using parameters m={m} and eps={eps} yields the next clustering results:')
    print()
    print(f'- Sil. score: {score}')
    print(f'- {instances} clustered instances into {clusters} clusters')
    print(f'- Avg. of {round(instances/clusters, 2)} instances per cluster')

In [ ]:
X[splits[0][0]].shape[0] * 0.85 # about 135 clustered instances are needed

#### Split #0

In [ ]:
X_split_0 = get_sim_mx_subset(X, splits[0][0])
X_split_0.shape

In [ ]:
dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_0, range(2, 8), range(10, 18))
dfs

In [ ]:
print_results(2, 11, dfs, dfi, dfc)

In [ ]:
clusterable_params.append([2, 11, 0])

#### Split #1

In [ ]:
X_split_1 = get_sim_mx_subset(X, splits[1][0])
X_split_1.shape

In [ ]:
dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_1, range(2, 8), range(10, 18))
dfs

In [ ]:
print_results(2, 11, dfs, dfi, dfc)

In [ ]:
clusterable_params.append([2, 11, 1])

#### Split #2

In [ ]:
X_split_2 = get_sim_mx_subset(X, splits[2][0])
X_split_2.shape

In [ ]:
dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_2, range(2, 8), range(10, 18))
dfs

In [ ]:
print_results(2, 11, dfs, dfi, dfc)

In [ ]:
clusterable_params.append([2, 11, 2])

#### Split #3

In [ ]:
X_split_3 = get_sim_mx_subset(X, splits[3][0])
X_split_3.shape

In [ ]:
dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_3, range(2, 8), range(10, 18))
dfs

In [ ]:
print_results(2, 11, dfs, dfi, dfc)

In [ ]:
clusterable_params.append([2, 11, 3])

#### Split #4

In [7]:
X_split_4 = get_sim_mx_subset(X, splits[4][0])
X_split_4.shape

NameError: name 'get_sim_mx_subset' is not defined

In [ ]:
dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_4, range(2, 8), range(10, 18))
dfs

In [ ]:
print_results(2, 11, dfs, dfi, dfc)

In [ ]:
clusterable_params.append([2, 11, 4])

#### Clusterable parameters for each split

In [ ]:
clusterable_params

## Clustering Results

In [ ]:
def get_indiv_clustering_results(params):
    '''Returns a dictionary mapping the name of an image
    with the cluster it belongs to'''
    # Preconditions
    split_idx = params[2]
    train_idxs = splits[split_idx][0]
    # Prepare data (always X, not feats_df)
    sim_mx_subset = get_sim_mx_subset(X, train_idxs)
    img_names = X_names[train_idxs]
    # Perform clustering
    dbscan = DBSCAN(min_samples=params[0], eps=params[1], metric='precomputed')
    dbscan = dbscan.fit(sim_mx_subset)
    # Generate {img_name : label} mapping
    name_label_map = {name: label for name, label in zip(img_names, dbscan.labels_)}
    return name_label_map

def get_global_clustering_results(params_set):
    '''Returns a dictionary mapping the index of every param set
    in 'params' arg. with the clustering results generated with that param. set'''
    results = {}
    for params in params_set:
        # Create { split_idx: cluster_labels} pair
        results[params[2]] = get_indiv_clustering_results(params)
    return results

In [ ]:
cl_results = get_global_clustering_results(clusterable_params)

In [ ]:
cl_results

In [ ]:
# Sanity check: Number of elements should be the same as clusters detected in clustering phase
for split_idx in cl_results.keys(): print(len(np.unique(list(cl_results[split_idx].values())))-1)

## Clustering Prototypes

In our experiment, we want to predict the vote count for a new image, based on the proximity it has to the avaliable clusters. These clusters are composed of many data points, so the proximity of a new data point to a cluster can be measured in different ways, like taking the distance between the new point and the nearest clustered point in the dataset.   
However, this approach can be biased when new poins get associated to the cluster taking in account the nearest point of a cluster instead of the overall position of a cluster. To avoid this, for each cluster we calculate a "prototype", a data point which is the centroid of all the data points in a cluster. This way, we can measure the distance to the general position of a cluster in a more confident way.

### Vote prototypes (Solution of cases)

In [48]:
def gen_indiv_vote_prototypes(cl_result, ignore_noise=True):
    # Separate image votes according to the clusters they belong to
    votes_by_cluster = {}
    for img_name, cl_idx in cl_result.items():
        if ignore_noise and cl_idx == -1: continue # ignore noise cluster
        img_votes = votes_df.loc[img_name].values[:-1]
        if cl_idx not in votes_by_cluster.keys(): votes_by_cluster[cl_idx] = [img_votes]
        else: votes_by_cluster[cl_idx].append(img_votes)
    # For each cluster, calculate their vote prototype
    vote_prts_by_cluster = {}
    for cl_idx, cl_votes in votes_by_cluster.items():
        unrounded_prt = np.average(np.array(cl_votes,'uint8'), axis=0)
        vote_prts_by_cluster[cl_idx] = np.array(np.round(unrounded_prt), 'int')
    return vote_prts_by_cluster
    
def get_global_vote_prototypes(cl_results, ignore_noise=True):
    global_vote_prototypes = {}
    for i, cl_result in cl_results.items():
        global_vote_prototypes[i] = gen_indiv_vote_prototypes(cl_result, ignore_noise=ignore_noise)
    return global_vote_prototypes

### Feature Prototypes (Description of cases)

In [ ]:
def gen_indiv_feat_prototypes(cl_result, ignore_noise=True):
    # Separate image votes according to the clusters they belong to
    feats_by_cluster = {}
    for img_name, cl_idx in cl_result.items():
        if ignore_noise and cl_idx == -1: continue # ignore noise cluster
        img_feats = votes_df.loc[img_name].values[:-1] # PENDING
        if cl_idx not in feats_by_cluster.keys(): feats_by_cluster[cl_idx] = [img_feats]
        else: feats_by_cluster[cl_idx].append(img_feats)
    # For each cluster, calculate their feature prototype
    feat_prts_by_cluster = {}
    for cl_idx, cl_feats in feats_by_cluster.items():
        unrounded_prt = np.average(np.array(cl_feats,'uint8'), axis=0)
        feat_prts_by_cluster[cl_idx] = np.array(np.round(unrounded_prt), 'int')
    return feat_prts_by_cluster
    
def get_global_feat_prototypes(cl_results, ignore_noise=True):
    global_feat_prototypes = {}
    for i, cl_result in cl_results.items():
        global__feat_prototypes[i] = gen_indiv_feat_prototypes(cl_result, ignore_noise=ignore_noise)
    return global_feat_prototypes

### Calculate both prototypes

In [49]:
global_vote_prototypes = get_global_vote_prototypes(cl_results)
global_feats_prototypes = get_global_feats_prototypes(cl_results)

In [50]:
# Sanity check: No. of elements should be the same as no. of clusters detected in clustering phase
global_vote_prototypes[3]

{0: array([5, 4, 4, 0]),
 1: array([7, 2, 5, 1]),
 2: array([8, 2, 3, 2]),
 3: array([ 8, 10,  2,  0]),
 4: array([7, 2, 6, 0]),
 5: array([8, 2, 2, 1]),
 6: array([7, 4, 2, 0]),
 7: array([9, 2, 1, 2]),
 8: array([6, 5, 3, 0]),
 9: array([6, 3, 4, 0]),
 10: array([6, 5, 2, 1]),
 11: array([11,  2,  6,  0]),
 12: array([8, 0, 4, 0]),
 13: array([5, 2, 5, 0]),
 14: array([8, 2, 2, 0])}

In [ ]:
# Sanity check: No. of elements should be the same as no. of clusters detected in clustering phase
global_feats_prototypes[3]

## Vote Count Prediction

In [51]:
def get_img_idxs_per_cluster(cl_results, ignore_noise=True):
    img_idxs_per_cluster = {}
    for img_name, cl_idx in cl_results.items():
        if cl_idx==-1 and ignore_noise: continue # ignore noise cluster
        img_idx = np.argwhere(X_names == img_name)[0][0]
        if cl_idx not in img_idxs_per_cluster.keys():
            img_idxs_per_cluster[cl_idx] = [img_idx]
        else:
            img_idxs_per_cluster[cl_idx].append(img_idx)
    return img_idxs_per_cluster

def get_dist_to_clusters(img_idx, img_idxs_per_cluster):
    dist_to_clusters = {}
    for cl_idx, img_idxs in img_idxs_per_cluster.items():
        distances = X[img_idx, img_idxs]
        dist_to_clusters[cl_idx] = np.average(distances)
    return dist_to_clusters

def get_nearest_clusters_indices(dist_to_clusters, k):
    if k >= len(dist_to_clusters): nearest_cls_idxs = list(dist_to_clusters.keys())
    else:
        nearest_cls_idxs = []
        for i in range(k): # K times...
            nearest_cl_idx, min_dist = None, np.inf
            # ...iterate searching the nearest cluster
            for cl_idx, dist in dist_to_clusters.items():
                if cl_idx in nearest_cls_idxs: continue # ignore prev. found nearest clusters
                if dist < min_dist: nearest_cl_idx, min_dist = cl_idx, dist
            nearest_cls_idxs.append(nearest_cl_idx)
    return nearest_cls_idxs

def get_indiv_vote_predictions(prototypes, cl_results, split_idx, k):
    vote_predictions = {}
    # Prepare data
    test_idxs = splits[split_idx][1]
    img_idxs_per_cluster = get_img_idxs_per_cluster(cl_results)
    # For each test image...
    for test_img_idx in test_idxs:
        # Measure average distances to each cluster
        dist_to_clusters = get_dist_to_clusters(test_img_idx, img_idxs_per_cluster)
        # Using those distances, find the nearest k clusters
        kn_clusters_idxs =  get_nearest_clusters_indices(dist_to_clusters, k=k)
        # Aggregate the vote prototypes of the clusters associated with those distances
        nearest_prototypes = [prototypes[kn_cl_idx] for kn_cl_idx in kn_clusters_idxs]
        unrounded_vcp = np.average(np.array(nearest_prototypes), axis=0)
        vote_count_prediction = np.round(unrounded_vcp)
        # Assoaciate name of imge with its vote prediction
        test_img_name = X_names[test_img_idx]
        vote_predictions[test_img_name] = vote_count_prediction
    return vote_predictions

def get_global_vote_predictions(global_prototypes, global_cl_results, k):
    global_vote_predictions = {}
    for split_idx in global_cl_results.keys():
        # Create { split_idx: vote_predictions } pairs
        global_vote_predictions[split_idx] = get_indiv_vote_predictions(global_prototypes[split_idx], global_cl_results[split_idx], split_idx, k=k)
    return global_vote_predictions

In [52]:
global_vote_predictions_k1 = get_global_vote_predictions(global_prototypes, cl_results, k=1)
global_vote_predictions_k3 = get_global_vote_predictions(global_prototypes, cl_results, k=3)
global_vote_predictions_k5 = get_global_vote_predictions(global_prototypes, cl_results, k=5)
global_vote_predictions_k7 = get_global_vote_predictions(global_prototypes, cl_results, k=7)

In [53]:
global_vote_predictions_k5

{0: {'2388889__hotdog__0.99999714.jpg': array([8., 3., 3., 1.]),
  '2417881__zebra__0.9999945.jpg': array([8., 3., 3., 1.]),
  '2403403__banana__0.9999926.jpg': array([7., 4., 2., 1.]),
  '2381941__zebra__0.9999914.jpg': array([8., 3., 3., 1.]),
  '2403741__zebra__0.99999523.jpg': array([7., 3., 4., 1.]),
  '2404281__zebra__0.999998.jpg': array([8., 3., 3., 1.]),
  '2416627__zebra__0.9999987.jpg': array([8., 3., 3., 1.]),
  '2391964__flamingo__1.0.jpg': array([8., 3., 3., 1.]),
  '2404583__umbrella__0.99999297.jpg': array([7., 3., 3., 1.]),
  '2409637__four-poster__0.99999464.jpg': array([7., 3., 4., 0.]),
  '2380669__parking_meter__0.9999993.jpg': array([7., 3., 4., 0.]),
  '2411196__crane__0.9999995.jpg': array([8., 3., 3., 1.]),
  '134__zebra__0.9999949.jpg': array([8., 3., 3., 1.]),
  '2405905__traffic_light__0.99999535.jpg': array([7., 3., 3., 1.]),
  '2404127__zebra__0.9999933.jpg': array([8., 3., 3., 1.]),
  '2406857__zebra__0.9999894.jpg': array([8., 3., 3., 1.]),
  '2414277__z

## Metric Evaluation

In [54]:
def calc_distance(p1, p2, metric):
    if metric == 'rmse': return np.sum(np.square(p1 - p2))
    elif metric == 'manhattan': return np.sum(np.abs(p1 - p2))
    else: print('Unknown metric type')

def eval_indiv_vote_preds(vote_predictions, metric):
    vote_distances = []
    for img_name, vote_pred in vote_predictions.items():
        # Fetch real votes and compare with vote predictions
        real_votes = votes_df.loc[img_name].values[:4]
        distance = calc_distance(real_votes, vote_pred, metric)
        vote_distances.append(distance)
    vote_distances = np.array(vote_distances)
    if metric=='rmse': metrics = {'rmse': round(np.sqrt(np.average(vote_distances)), 2)}
    elif metric=='manhattan': metrics = {'manhattan': (np.average(vote_distances), 2)}
    else:
        metrics = {
            'average': round(np.average(vote_distances), 2),
            'std. dev.': round(np.std(vote_distances), 2),
            'range': [round(np.min(vote_distances), 2), round(np.max(vote_distances), 2)],
        }
    return metrics

def eval_global_vote_preds(global_vote_predictions, metric):
    global_metrics = {}
    # Calculate metrics for each split
    for split_idx, vote_predictions in global_vote_predictions.items():
        global_metrics[split_idx] = eval_indiv_vote_preds(vote_predictions, metric)
    # Aggregate metrics for all splits
    global_metrics['global'] = {}
    for metric_type in global_metrics[0].keys():
        metrics_per_type = [metrics[metric_type] for split_key, metrics in global_metrics.items() if split_key != 'global']
        avgd_metrics_per_type = np.round(np.average(np.array(metrics_per_type), axis=0), 2)
        if metric_type == 'range': avgd_metrics_per_type = list(avgd_metrics_per_type)
        global_metrics['global'][metric_type] = avgd_metrics_per_type
    return global_metrics

In [55]:
METRIC = 'rmse'
global_vote_metrics_k1 = eval_global_vote_preds(global_vote_predictions_k1, metric=METRIC)
global_vote_metrics_k3 = eval_global_vote_preds(global_vote_predictions_k3, metric=METRIC)
global_vote_metrics_k5 = eval_global_vote_preds(global_vote_predictions_k5, metric=METRIC)
global_vote_metrics_k7 = eval_global_vote_preds(global_vote_predictions_k7, metric=METRIC)

In [56]:
global_vote_metrics_k1['global']

{'rmse': 4.08}

In [57]:
global_vote_metrics_k3['global']

{'rmse': 4.24}

In [58]:
global_vote_metrics_k5['global']

{'rmse': 4.34}

In [59]:
global_vote_metrics_k7['global']

{'rmse': 4.44}

The previous results shine a light about the viability to predict the vote count for a new image given the vote prototypes of previously generated image clusters.   

In average, the predicted vote count for a new image differs by 6 votes compared to the real vote count. The difference between vote count shows a ascending tendence proportional to the number of nearest clusters used in the vote count prediction, although the growing rate is very small. In the end, this means that when predicting the vote count for a new image, it is recommended to use the vote prototype of only the nearest cluster.   

Additional metrics also show that the distribution of vote count differences shows a gaussian shape with a slight skeweness to the right, i.e. towards higher vote differences). The standard deviation shows that the majority of vote differences are between +-3 to the average vote difference. Given that the average vote difference is 6, this means that the majority of vote differences will be inside the 3-9 range.

Taking in account that for every image around 30 votes were casted, the difference in vote count prediction is pretty large. A difference of 6 votes when predicting votes can be really important. However, we need to calculate vote count proportion differences, beacuase, at the end of the day, proportions are also a important factor in deciding which techniques are better for new images.